In [0]:
from pyspark.sql.functions import *
import urllib

delta_path = "dbfs:/user/hive/warehouse/authentication_credentials"
credentials_df = spark.read.format("delta").load(delta_path)

ACCESS_KEY = credentials_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = credentials_df.select('Secret access key').collect()[0]['Secret access key']

ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

AWS_S3_BUCKET = "user-0eaf46a0829f-bucket"
MOUNT_NAME = "/mnt/Ben_data_mount"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)



In [0]:
location_of_pinfile = "s3://user-0eaf46a0829f-bucket/topics/0eaf46a0829f.pin/partition=0/*.json"

file_type = "json"
infer_schema = "true"

df_pin = spark.read.format(file_type).option("inferSchema", infer_schema).load(location_of_pinfile)

display(df_pin)

In [0]:
#This cell here 
file_location = "s3://user-0eaf46a0829f-bucket/topics/0eaf46a0829f.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_geo = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
# Display Spark dataframe to check its content
display(df_geo)

In [0]:
df_geo.printSchema()

In [0]:


location_of_userfile = "s3://user-0eaf46a0829f-bucket/topics/0eaf46a0829f.user/partition=0/*.json"

file_type = "json"
infer_schema = "true"

df_user = spark.read.format(file_type).option("inferSchema", infer_schema).load(location_of_userfile)

display(df_user)

In [0]:
df_user.printSchema()

In [0]:
df_pin.printSchema()

In [0]:

from pyspark.sql.functions import regexp_replace


def clean_data(df_pin):
    cleaned_df = df_pin.replace({'No description available Story format': None,}, subset=['description'])
    cleaned_df = cleaned_df.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None,}, subset=['tag_list'])
    cleaned_df = cleaned_df.replace({'No Title Data Available': None,}, subset=['title'])
    cleaned_df = cleaned_df.replace({'User Info Error': None, }, subset=['follower_count'])
    cleaned_df = cleaned_df.replace({'Image src error.': None, }, subset=['image_src'])
    cleaned_df = cleaned_df.replace({'User Info Error': None, }, subset=['poster_name'])

    cleaned_data_df = cleaned_df.withColumn('follower_count', cleaned_df['follower_count'].cast('int'))
    cleaned_data_df = cleaned_df.withColumn('index', cleaned_df['index'].cast('int'))
    cleaned_data_df = cleaned_data_df.withColumn('save_location', regexp_replace('save_location', 'Local save in', ''))
    cleaned_data_df = cleaned_data_df.withColumn('follower_count', regexp_replace('follower_count', 'k', '000'))
    cleaned_data_df = cleaned_data_df.withColumn('follower_count', regexp_replace('follower_count', 'M', '000000'))

    cleaned_data_df = cleaned_data_df.withColumnRenamed('index', 'ind')
    cleaned_data_df = cleaned_data_df.select('ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category')

    return cleaned_data_df




In [0]:
cleaned_pin = clean_data(df_pin)
display(cleaned_pin)

In [0]:
cleaned_pin.printSchema()

In [0]:
from pyspark.sql.functions import array, col, to_timestamp

def cleaned_geo(df_geo):
    clean_df = df_geo.withColumn('coordinates', array(col('longitude'), col('latitude')))
    clean_df = clean_df.drop('longitude', 'latitude')

    clean_df = clean_df.withColumn('timestamp', to_timestamp(col('timestamp')))
    clean_df = clean_df.select('ind', 'country', 'coordinates', 'timestamp')

    return clean_df

In [0]:
cleaned_geo_df = cleaned_geo(df_geo)
display(cleaned_geo_df)

In [0]:
cleaned_geo_df.printSchema()

In [0]:
from pyspark.sql.functions import concat

def cleaned_user(df_user):
    clean_user_df = df_user.withColumn('user_name', concat('first_name', 'last_name'))
    clean_user_df = clean_user_df.drop('first_name', 'last_name')

    clean_user_df = clean_user_df.withColumn('date_joined', to_timestamp(col('date_joined')))
    clean_user_df = clean_user_df.select('ind' , 'user_name', 'age', 'date_joined')
    return clean_user_df

In [0]:
cleaned_user = cleaned_user(df_user)
display(cleaned_user)

In [0]:
cleaned_user.printSchema()

In [0]:
from pyspark.sql.functions import count
from pyspark.sql.window import Window

joined_df = cleaned_pin.join(cleaned_geo_df, cleaned_pin["ind"] == cleaned_geo_df["ind"], how="inner")

people_cat = Window.partitionBy("category").orderBy("country")
the_result = joined_df.withColumn("category_count", count("*").over(people_cat))

the_result = the_result.select("category", "country", "category_count").distinct()
display(the_result)



In [0]:
from pyspark.sql.functions import count, year, col, asc


joined_df = cleaned_pin.join(cleaned_geo_df, cleaned_pin["ind"] == cleaned_geo_df["ind"], how="inner")
years_df = joined_df.withColumn("post_year", year(col("timestamp")))
year_cat = years_df.filter(col("post_year").between(2018, 2022))

result_df = years_df.groupBy("category", "post_year").agg(count("*").alias("category_count"))

display(result_df)



In [0]:
from pyspark.sql.functions import row_number, desc
from pyspark.sql.window import Window

joined_df = cleaned_pin.join(cleaned_geo_df, cleaned_pin["ind"] == cleaned_geo_df["ind"], how="inner")

user_df = Window.partitionBy("country").orderBy(desc("follower_count"))
maxed_df = joined_df.withColumn("rank", row_number().over(user_df))

country_rank = maxed_df.filter(maxed_df.rank == 1)
result_df = country_rank.select("country", "poster_name", "follower_count")

max_result_df = result_df.orderBy(desc("follower_count")).limit(1)
max_results_df = max_result_df.select("country", "follower_count")

display(max_results_df)

In [0]:
from pyspark.sql.functions import count, col

joined_df = cleaned_pin.join(cleaned_user, cleaned_pin["ind"] == cleaned_user["ind"], how="inner")
age_df = joined_df.withColumn("age_group", when(col("age").between(18, 24), "18-24").when(col("age").between(25, 35), "25-35").when(col("age").between(36, 50), "36-50").when(col("age")> 50, "50+"))
age_cat = age_df.groupBy("age_group", "category").agg(count("*").alias("category_count"))
age_cat = age_cat.select("age_group", "category", "category_count")
display(age_cat)
# we need to get max category per age_group, so we'll want to use the partition by age_group so that we can then order each category count in descending order filtered by age group.
w = Window.partitionBy("age_group").orderBy(desc("category_count"))

# We can now use the row_number function to get the first row in each partition, and then filter to only keep the first row
age_cat = age_cat.withColumn("row", row_number().over(w)).filter(col("row") == 1).drop("row")
display(age_cat)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, asc, row_number

joined_df = cleaned_pin.join(cleaned_user, cleaned_pin["ind"] == cleaned_user["ind"], how="inner")
age_df = joined_df.withColumn("age_group", when(col("age").between(18, 24), "18-24") \
                              .when(col("age").between(25, 35), "25-35") \
                              .when(col("age").between(36, 50), "36-50") \
                              .when(col("age")> 50, "50+")) \
                              .orderBy(asc("age_group"))
median_age = age_df.groupBy("age_group").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))


display(median_age)

In [0]:
from pyspark.sql.functions import countDistinct, year, col

joined_data = cleaned_geo_df.join(cleaned_user, cleaned_geo_df["ind"] == cleaned_user["ind"], how="inner")

user_data = joined_data.withColumn("post_year", year(col("timestamp")))

joined_year = user_data.filter(col("post_year").between(2015, 2020))

users_by_year = joined_year.groupBy("post_year").agg(countDistinct("user_name").alias("number_users_joined"))

display(users_by_year.orderBy("post_year"))


In [0]:
from pyspark.sql.functions import countDistinct, year, col

joined_df = cleaned_pin.join(cleaned_user, cleaned_pin["ind"] == cleaned_user["ind"], how="inner")
final_join = joined_df.join(cleaned_geo_df, on="ind", how="inner")

user_follow = final_join.withColumn("post_year", year(col("timestamp")))
user_follow_year = user_follow.filter(col("post_year").between(2015, 2020))

user_follow_count = user_follow_year.groupBy("post_year").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))


display(user_follow_count.orderBy("post_year"))


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import year, col

joined_df = cleaned_pin.join(cleaned_user, on="ind", how="inner")
final_join = joined_df.join(cleaned_geo_df, on="ind", how="inner")

user_date = final_join.withColumn("post_year", year(col("timestamp"))) \
                      .withColumn("age_group", when(col("age").between(18, 24), "18-24") \
                                  .when(col("age").between(25, 35), "25-35") \
                                  .when(col("age").between(36, 50), "36-50") \
                                  .when(col("age")> 50, "50+")) \
                                  .orderBy(asc("age_group"))
user_date_year = user_date.filter(col("post_year").between(2015, 2020))


user_date_count = user_date_year.groupBy("age_group", "date_joined").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
user_date_count = user_date_count.select("age_group", "date_joined", "median_follower_count")

display(user_date_count)






